In [ ]:
import os
import glob
import json
import numpy as np
import copy
import random
import pdb
import trimesh
from utils.trans import *
from utils.utils import *
from PIL import Image
import shutil
import torch
import open3d as o3d
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# import os
# import json

# base_path = '/home/lidosan/Datasets/datasets--ShapeNet--PartNet-archive/blobs/data_v0/'

# directories = os.listdir(base_path)
# obj_correspondence = {}

# for obj_id in directories:
#     dir_path = os.path.join(base_path, obj_id)
#     meta_file = os.path.join(dir_path, 'meta.json')
#     parts_render_file = os.path.join(dir_path, 'parts_render', '0.png')
    
#     if os.path.isfile(meta_file) and os.path.isfile(parts_render_file):
#         with open(meta_file, 'r') as f:
#             meta_data = json.load(f)
        
#         category = meta_data.get('model_cat')
        
#         if category not in obj_correspondence:
#             obj_correspondence[category] = {}
        
#         obj_correspondence[category][obj_id] = meta_file

# # Write the obj_correspondence dictionary to a JSON file
# output_json_path = os.path.join('', 'obj_correspondence.json')
# with open(output_json_path, 'w') as outfile:
#     json.dump(obj_correspondence, outfile, indent=4)

# print("Organizing and copying completed successfully.")


In [ ]:
import os
import json

base_path = '/home/lidosan/Datasets/datasets--ShapeNet--PartNet-archive/blobs/data_v0/'
obj_ids = ["46531", "12159", "2554", "2355", "18233", "18277", "22981", "45187","9067","3512","3810","15719","10207","6490","8795",'3444','4739','7688']

directories = os.listdir(base_path)
obj_correspondence = {}

for obj_id in obj_ids:
    dir_path = os.path.join(base_path, obj_id)
    meta_file = os.path.join(dir_path, 'meta.json')
    parts_render_file = os.path.join(dir_path, 'parts_render', '0.png')
    
    if os.path.isfile(meta_file) and os.path.isfile(parts_render_file):
        with open(meta_file, 'r') as f:
            meta_data = json.load(f)
        
        category = meta_data.get('model_cat')
        
        if category not in obj_correspondence:
            obj_correspondence[category] = {}
        
        obj_correspondence[category][obj_id] = meta_file

# Write the obj_correspondence dictionary to a JSON file
output_json_path = os.path.join('', 'obj_correspondence_tiny.json')
with open(output_json_path, 'w') as outfile:
    json.dump(obj_correspondence, outfile, indent=4)

print("Organizing and copying completed successfully.")

In [ ]:
def handling_the_label(category,obj_structure):
    parts = extract_parts_v2(obj_structure)
    #TODO merge and extract the sub category
    if category in ['StorageFurniture','Table','Table','Bed','Earphone','Faucet']:
   
        pass
    if category in ['StorageFurniture']:
            #TODO Select the cabniet
            sub_category = parts[0][0].split('_')[1].strip()
            if sub_category == 'Cabinet':
                for part in parts:
                    print(part)
                    if 'Sink' in part[0]:
                        sub_category = 'Sink Cabinet'
                        print(sub_category)
                        break
                    elif 'Mirror' in part[0]:
                        sub_category = 'Mirror Cabinet'
                        print(sub_category)
                        break
                    elif 'Book' in part[1]:
                        sub_category = 'Book Shelf'
                        print(sub_category)
                        break

    if category in ['Chair']:
            # Extract parts with labels and object files
            for part in parts:
                if "Sofa-style" in part[0]:
                    sub_category = 'Sofa Chair'
                    print(sub_category)
                    break
                elif 'Star' in part[0]:
                    sub_category = 'Office Chair'
                    print(sub_category)
                    break
                sub_category = 'Regular Chair'
    return category,sub_category
    

In [ ]:
len(trimesh.load('scene_2b_colored/mesh_trans/Bed_caa_12159.obj').vertices)

In [ ]:
import trimesh
import numpy as np
from scipy.spatial import cKDTree

# # Load your mesh
# mesh = trimesh.load('path/to/your/mesh.obj')

# Generate random labels for each vertex
vertex_labels = np.random.randint(0, 1000, size=len(mesh.vertices))

# Function to downsample vertices
def downsample_mesh(mesh, target_num_vertices):
    # Use Trimesh's built-in simplification (decimation) method
    # The function simplifies the mesh to a target number of faces, not vertices
    # So we need to estimate the number of faces that would approximately give us the target number of vertices
    target_num_faces = int(target_num_vertices * (len(mesh.faces) / len(mesh.vertices)))
    
    # Simplify the mesh
    simplified_mesh = mesh.simplify_quadratic_decimation(target_num_faces)
    
    # Return the simplified mesh
    return simplified_mesh

# # Target number of vertices
# target_num_vertices = 1000  # Set your desired number of vertices

# # Downsample the mesh
# simplified_mesh = downsample_mesh(mesh, target_num_vertices)

# # Create a KDTree for the original and simplified vertices
# original_vertices = mesh.vertices
# simplified_vertices = simplified_mesh.vertices

# tree = cKDTree(original_vertices)

# # Find the nearest original vertex for each simplified vertex
# _, indices = tree.query(simplified_vertices)

# # Map the labels to the simplified vertices
# simplified_vertex_labels = vertex_labels[indices]

# # Now `simplified_mesh` has the downsampled vertices
# # and `simplified_vertex_labels` contains the labels for the simplified vertices

# # Save the simplified mesh if needed
# simplified_mesh.export('path/to/simplified_mesh.obj')

# # Print out the simplified vertices and their labels
# for i, vertex in enumerate(simplified_vertices):
#     print(f"Vertex: {vertex}, Label: {simplified_vertex_labels[i]}")


In [ ]:
def downsample_mesh(mesh, target_num_vertices):
    # Use Trimesh's built-in simplification (decimation) method
    # The function simplifies the mesh to a target number of faces, not vertices
    # So we need to estimate the number of faces that would approximately give us the target number of vertices
    target_num_faces = int(target_num_vertices * (len(mesh.faces) / len(mesh.vertices)))
    
    # Simplify the mesh
    simplified_mesh = mesh.simplify_quadratic_decimation(target_num_faces)
    
    # Return the simplified mesh
    return simplified_mesh

In [ ]:
obj_base_dir = f'/home/lidosan/Datasets/datasets--ShapeNet--PartNet-archive/blobs/data_v0/46531'
obj_structure = load_json(os.path.join(obj_base_dir, 'result.json'))
model_cat = load_json(os.path.join(obj_base_dir, 'meta.json'))['model_cat']
parts_render_img = os.path.join(obj_base_dir, 'parts_render', '0.png')
# Extract parts with labels and object files
parts = extract_parts_v2(obj_structure)
parts

In [3]:
def load_parts2obj_uv(obj_id, parts, label_to_index):
    obj_base_dir = f'/home/lidosan/Datasets/datasets--ShapeNet--PartNet-archive/blobs/data_v0/{obj_id}/objs'
    
    combined_mesh_list = []
    combined_vertices = []
    combined_labels = []
    obbs_dict = {}
    count_dict = {}
    unique_labels = {label for label, _ in parts}

    label_counts = {label: 0 for label in unique_labels}
    unique_labels_mesh_dict = {label: [] for label in unique_labels}

    for label, objs in parts:
        label_number = label_to_index.get(label)
        if label_number is None:
            print(f"Label {label} not found in label_to_index")
            continue
        
        for obj in objs:
            mesh_path = os.path.join(obj_base_dir, f"{obj}.obj")
            if os.path.exists(mesh_path):
                mesh = trimesh.load(mesh_path)
                unique_node_name = f"{label}_{label_counts[label]}"
                unique_node_name = unique_node_name.split('/')[0] + ':' + unique_node_name.split('/')[-1]
                obbs_dict[unique_node_name] = mesh.bounds
                count_dict[unique_node_name] = mesh.vertices.shape[0]
                unique_labels_mesh_dict[label].append(mesh)
                label_counts[label] += 1

    for label, meshes in unique_labels_mesh_dict.items():
        label_number = label_to_index.get(label)
        if label_number is None:
            print(f"Label {label} not found in label_to_index")
            continue
            
        combined_mesh = trimesh.util.concatenate(meshes)
        if len(combined_mesh.vertices) > 10000:
            save_as_uv_mesh(downsample_mesh(combined_mesh, 10000), 'temp.obj')
        else:
            save_as_uv_mesh(combined_mesh, 'temp.obj')
        
        combined_mesh_uv = trimesh.load('temp.obj')
        if len(combined_mesh.vertices) > 1024:
            trimesh.grouping.merge_vertices(combined_mesh_uv, merge_tex=True)

        vertices = combined_mesh_uv.vertices
        num_vertices = vertices.shape[0]
        labels = np.full((num_vertices, 1), label_number)

        combined_vertices.append(vertices)
        combined_labels.append(labels)
        combined_mesh_list.append(combined_mesh_uv)
        
    if combined_vertices:
        combined_vertices = np.vstack(combined_vertices)
        combined_labels = np.vstack(combined_labels)
        point_cloud = np.hstack((combined_vertices, combined_labels))
    else:
        point_cloud = np.array([])

    final_mesh = trimesh.util.concatenate(combined_mesh_list)
    print(label,len(final_mesh.vertices), point_cloud.shape)
    
    return point_cloud, final_mesh, obbs_dict

In [ ]:
import os
import json
import numpy as np
import shutil
import re
import torch
# Assuming the following functions are defined:
# load_json, extract_parts, load_parts2obj_uv

obj_ids = ["46531", "12159", "2554", "2355", "18233", "18277", "22981", "45187","9067","3512","3810","15719","10207","6490","8795",'3444','4739']
target_dir = '/home/lidosan/Datasets/PartNet_complete'

label_to_index = load_json('label2idx.json')


def clean_label(label):
    """Remove specific substrings from the label."""
    cleaned_label = re.sub(r'\(other\)/other_leaf|\(other\)|leaf', '', label)
    return cleaned_label.strip()

def normalize_label(label):
    """Convert the label to the desired format and clean it."""
    replacements = {
        " ": "_",
        "/other_": "",
        "sitting_furniture": "chair",
        "pot_body": "body",
        "-": "_",
        "cylinder": "decoration",
        # "/decoration": "",
        "star_shape": "star",
        "sofa_style_arm": "arm_sofa_style",
        "stuffs_contained_in_the_pot":"containing_things",
        "display/screen":"display/display_screen",
        "mug/stuffs_contained":"mug/containing_things"
    }
    
    cleaned_label = clean_label(label).lower()

    for old, new in replacements.items():
        cleaned_label = cleaned_label.replace(old, new)

    if "chair/chair_base/" in cleaned_label:
        cleaned_label = "chair/chair_base"
    elif "storage_furniture/cabinet/cabinet_frame" in cleaned_label and "bar" in cleaned_label:
        # merge the storage_furniture_bar
        cleaned_label = "storage_furniture/cabinet/cabinet_frame"
    return cleaned_label



def extract_parts_v2(data, parent_label=''):
    parts = []
    if isinstance(data, dict):
        if 'text' in data:
            label = parent_label + '/' + data['text'] if parent_label else data['text']
            label = normalize_label(label)
            if 'objs' in data:
                parts.append((label, data['objs']))
            if 'children' in data:
                for child in data['children']:
                    parts.extend(extract_parts_v2(child, label))
    elif isinstance(data, list):
        for item in data:
            parts.extend(extract_parts_v2(item, parent_label))
    return parts



for obj_id in obj_ids:
    number_to_label = {}
    obj_base_dir = f'/home/lidosan/Datasets/datasets--ShapeNet--PartNet-archive/blobs/data_v0/{obj_id}'
    
    obj_structure = load_json(os.path.join(obj_base_dir, 'result.json'))
    model_cat = load_json(os.path.join(obj_base_dir, 'meta.json'))['model_cat']
    parts_render_img = os.path.join(obj_base_dir, 'parts_render', '0.png')
    # Extract parts with labels and object files
    parts = extract_parts_v2(obj_structure)
    # # Load parts and create the mesh
    points, combined_mesh,obbs = load_parts2obj_uv(obj_id, parts, label_to_index)
    
    
    
    obj_path = os.path.join(target_dir, obj_id)
    # if os.path.exists(obj_path):
    #     continue
    os.makedirs(obj_path, exist_ok=True)
    texture_dir = os.path.join(obj_path,'Texture')
    # Export mesh
    combined_mesh.export(os.path.join(obj_path, f'{obj_id}.obj'))
    
    #Todo
    
    # os.makedirs(texture_dir)
    # Remove unnecessary files
    mtl_file = os.path.join(obj_path, 'material.mtl')
    png_file = os.path.join(obj_path, 'material_0.png')
    output_json_path = os.path.join(obj_path,'meta.json')
    if os.path.exists(mtl_file):
        os.remove(mtl_file)
    if os.path.exists(png_file):
        os.remove(png_file)
    
    # Export segmentation labels as txt
    seg_label = points[:, 3]  # Assuming `pcd` has the point cloud data
    np.savetxt(os.path.join(obj_path, f'seg_label.txt'), seg_label, fmt='%d')
    np.save(os.path.join(obj_path, f'points.npy'),points[:,:3])
    # # Export label to number mapping as json
    # with open(os.path.join(obj_path, f'label_to_number.json'), 'w') as f:
    #     json.dump(number_to_label, f, indent=4)
    
    # Copy the render image to the new directory
    new_image_path = os.path.join(obj_path, 'parts_render_0.png')
    shutil.copy(parts_render_img, new_image_path)
    
    
    # Save label_to_number dictionary as .pth file using torch.save
    torch.save(obbs, os.path.join(obj_path, 'obbs.pth'))
    
    # save npy

    # Save the meta result json:
    # os.makedirs(obj_path, exist_ok=True)

    #TODO use Meshy api to gain the image
    with open(output_json_path, 'w') as outfile:
        json.dump({'model_cat':model_cat}, outfile, indent=4)

    print(model_cat)
    
print("Export completed successfully.")

In [ ]:
'storage_furniture/cabinet/object/book'.replace('storage_furniture/cabinet/object', 'decoration')

In [ ]:
# case 1 down sample each part and concat together (accurate everything, but large size still)


# case 2 down sample the mesh, and approxmatly estimate the uv and label(doing)


number_to_label

In [ ]:
def load_seg(filename):
    """Read and return the contents of the specified file as a list of integers."""
    with open(filename, 'r') as file:
        lines = file.readlines()
        return [int(line.strip()) for line in lines]

In [ ]:
id='46531'
mesh = trimesh.load(f'/home/lidosan/Datasets/PartNet_complete/{id}/{id}.obj')
label = np.asarray(load_seg(f'/home/lidosan/Datasets/PartNet_complete/{id}/seg_label.txt'))
poitns = np.asarray(mesh.vertices)

In [ ]:
# mesh = trimesh.load('path/to/your/mesh.obj')
import numpy as np
from scipy.spatial import cKDTree

# Generate random labels for each vertex
vertex_labels = label
# vertex_uvs = np.asarray(mesh.visual.uv)
# Function to downsample vertices
def downsample_mesh(mesh, target_num_vertices):
    # Use Trimesh's built-in simplification (decimation) method
    # The function simplifies the mesh to a target number of faces, not vertices
    # So we need to estimate the number of faces that would approximately give us the target number of vertices
    target_num_faces = int(target_num_vertices * (len(mesh.faces) / len(mesh.vertices)))
    
    # Simplify the mesh
    simplified_mesh = mesh.simplify_quadratic_decimation(target_num_faces)
    
    # Return the simplified mesh
    return simplified_mesh

# Target number of vertices
target_num_vertices = 8000  # Set your desired number of vertices

# Downsample the mesh
simplified_mesh = downsample_mesh(mesh, target_num_vertices)

# Create a KDTree for the original and simplified vertices
original_vertices = mesh.vertices
simplified_vertices = simplified_mesh.vertices

tree = cKDTree(original_vertices)

# Find the nearest original vertex for each simplified vertex
_, indices = tree.query(simplified_vertices)

# Map the labels to the simplified vertices
simplified_vertex_labels = vertex_labels[indices]

In [ ]:
# Extract coordinates and labels
verts = simplified_vertices
labels = simplified_vertex_labels
# Normalize labels for coloring
max_label = labels.max()
colors = plt.get_cmap("viridis")(labels / max_label)[:, :3]

# # Create Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(verts)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd], window_name="Point Cloud with Segmentation Labels", width=800, height=600)

In [ ]:
mesh = o3d.io.read_triangle_mesh('/home/lidosan/Workplace-why/OpenIns3D/demo/demo_scene/scannet/scannet_scene2.ply.ply')
o3d.visualization.draw_geometries([mesh], window_name="Point Cloud with Segmentation Labels", width=800, height=600)

In [ ]:
mesh

In [ ]:
trimesh.load('colored_scene/combined_mesh.ply').show(viewer='gl')

# Generate Secne test

In [ ]:
from utils.trans import  * 
# Define the InstanceObject class
class InstanceObject_v2:
    def __init__(self, obj_id, category, instance_mesh, scale_factor, obbs, orientation=np.array([0, 1, 0])):
        self.obj_id = obj_id
        self.category = category
        self.mesh = basic_transformation(instance_mesh, scale_factor)
        self.scale_factor = scale_factor
        self.orientation = orientation

        # Initialize position and transformation attributes
        self.position = np.zeros(3)
        self.rotation_matrix = np.eye(4)
        self.translation_matrix = np.eye(4)
        # Extract mesh details
        # self.contains = list(self.mesh.geometry.keys())
        self.obbs = obbs
        self.central_points = {}
        self.bounds = self.mesh.bounds
        self.height = self.bounds[1][2] - self.bounds[0][2]
        self.part_list = []  # store all the parts information

        # Additional attributes
        self.rotation = 0
        self.transformation = None
        self.objects_on_surface = {}  # Objects on the surfaces


    def apply_transformation(self):
        final_translation_matrix = create_translation_matrix(self.position)
        center_translation_to_origin = create_translation_matrix(-self.mesh.centroid)
        center_translation_back = create_translation_matrix(self.mesh.centroid)
        final_transform = np.dot(
            center_translation_back,
            np.dot(create_rotation_matrix('z', self.rotation), center_translation_to_origin)
        )
        final_transform = np.dot(final_translation_matrix, final_transform)
        self.mesh.apply_transform(final_transform)
        self.transformation = final_transform
        return final_transform

    def get_surface_mesh(self):
        surface_nodename_map = {
            'Bed': ' Mattress_0',
            'Table': ' Board_0'
        }
        surface_nodename = surface_nodename_map[self.category]
        transform = self.mesh.graph[surface_nodename][0]
        top_mesh = copy.deepcopy(self.mesh.geometry[surface_nodename])
        top_mesh.apply_transform(transform)
        return top_mesh

    def overlap_detection(self, other_mesh):
        other_bbox = other_mesh.bounds
        for node_name in self.mesh.graph.nodes_geometry:
            geometry = self.mesh.geometry[node_name]
            transform = self.mesh.graph[node_name][0]
            geometry_copy = copy.deepcopy(geometry)
            geometry_copy.apply_transform(transform)
            self_bbox = geometry_copy.bounds
            intersects = not (
                self_bbox[1][0] < other_bbox[0][0] or self_bbox[0][0] > other_bbox[1][0] or  # x-axis
                self_bbox[1][1] < other_bbox[0][1] or self_bbox[0][1] > other_bbox[1][1] or  # y-axis
                self_bbox[1][2] < other_bbox[0][2] or self_bbox[0][2] > other_bbox[1][2]    # z-axis
            )
            if intersects:
                print('Overlap')
                return True
        return False


In [ ]:
import os
import json
import numpy as np
import copy
import random
import trimesh
from utils.utils import *
from utils.trans  import InstanceObject, place_object_on, basic_transformation
import shutil

# surface_top_list = ['Mug','Hat','Bowl||','Lamp','Vase','Laptop','Keyboard','Bag','Scissors','Display','Knife','Bottle','Earphone']

# obejct_with_top_surface_list = ['StorageFurniture','Refrigerator', 'Table','Bed', 'Dishwasher']

# obejct_without_top_surface_list =['TrashCan','Chair','Door']



surface_top_list = ['Mug','Lamp','Vase','Laptop','Keyboard','Bag','Display','Bottle']
# No Knife,Hat,'Display','Scissors','Earphone','Bowl' for now 

obejct_with_top_surface_list = ['StorageFurniture','Refrigerator', 'Table','Bed', 'Dishwasher']

obejct_without_top_surface_list =['TrashCan','Chair','Door']





In [ ]:
cat_factors = {
    'Mug': (0.2, 0.3),        # Mugs typically vary a bit in size but are generally small.
    'Hat': (0.2, 0.3),        # Hats have some variation but are generally smaller.
    'Bowl': (0.2, 0.3),       # Bowls can range from small to medium sizes.
    'Lamp': (0.4, 0.5),       # Lamps can range from small desk lamps to larger ones.
    'Vase': (0.4, 0.5),       # Vases can vary significantly in size but are usually not as large as laptops.
    'Laptop': (0.8, 1.0),     # Laptops have a narrow size range and are relatively larger.
    'Keyboard': (0.8, 1.0),   # Keyboards also have a narrow size range similar to laptops.
    'Bag': (0.5, 1.0),        # Bags can vary significantly in size.
    'Scissors': (0.2, 0.5),   # Scissors typically have a limited size range and are generally small.
    'Display': (0.7, 1.0),    # Displays usually have a narrow size range and are relatively larger.
    'Knife': (0.2, 0.5),      # Knives can vary from small to medium sizes.
    'Bottle': (0.2, 0.5),     # Bottles can vary significantly in size.
    'Earphone': (0.2, 0.5)    # Earphones are generally very small.
}


In [ ]:
config = load_json('scene_layout.json')
all_obj_dict = load_json('obj_correspondence_tiny.json')
large_id_list = [obj['obj_id'] for obj in config['large_objects']]
large_objects_contain = {}
tiny_obj_id_list = []
obj_num = 1
for obj_id in [obj['obj_id'] for obj in config['large_objects']]:
    tiny_objects = []
    for i in range(obj_num):
        # still random
        tiny_obj_cat, tiny_obj_id= random_objects_generate(surface_top_list,all_obj_dict)
        tiny_obj_info = {
            "obj_id":tiny_obj_id,
            "category":tiny_obj_cat,
            "scale_factor":random.uniform(cat_factors[tiny_obj_cat][0],cat_factors[tiny_obj_cat][1])
        }
        tiny_objects.append(tiny_obj_info)
        tiny_obj_id_list.append(tiny_obj_id)
    large_objects_contain[obj_id] = tiny_objects
id_list = large_id_list+tiny_obj_id_list

# 读取所有的点云


In [ ]:
torch.load('')

In [ ]:
def generate_scene_object_v2(id_list): 
    point_clouds = {}
    mesh_dict = {}
    label_to_number = {}
    cat_to_id = {}
    for obj_id in id_list:
        # print(obj_id)
        obj_base_dir = '/home/lidosan/Datasets/PartNet_complete/{}'.format(obj_id)
        obj_structure = load_json(os.path.join(obj_base_dir, 'label_to_number.json'))
        # obj_infromation = load_json(os.path.join(obj_base_dir, 'meta.json'))
        # Extract parts with labels and object files
        parts = extract_parts(obj_structure)

        # Load first object
        point_cloud, label_to_number, combined_mesh = load_and_color_meshes_semantics(obj_id, parts, label_to_number)

        mesh_dict[obj_id] = combined_mesh
        point_clouds[obj_id] = point_cloud
        cat_to_id[obj_id] = obj_infromation['model_cat']
    return point_clouds, mesh_dict, label_to_number,cat_to_id


In [ ]:
load_json(os.path.join(obj_base_dir, 'meta.json'))

In [ ]:
import webbrowser
print("Make sure your file's visibility "+'in Google Drive is set to "Anyone with the link"')
input_link = 'https://drive.google.com/file/d/1Y3R1nu1uO66VD4RmXHGhiKMqztCnEkOK/view?usp=drive_link'
input_link = input_link[32:]
end = input_link.index('/')
ID = input_link[:end]
download_link = 'https://drive.google.com/uc?export=download&id='+ID
print('This Your Direct Download Link : ',download_link)
# ok = False
# while not(ok) :
#     prompt = input('You Wanna Download The File Right Now ?  Y/N : ')
#     ok = prompt =='Y' or prompt =='N' or prompt =='n' or prompt =='y'

prompt = 'N'
if prompt == 'Y' or prompt == 'y'  :
    webbrowser.open(download_link)
    
else :
    print('thank u for your time <3 ')

In [ ]:
download_link


In [ ]:
from urllib.request import urlopen 
download_link = 'https://drive.google.com/uc?export=download&id=1Y3R1nu1uO66VD4RmXHGhiKMqztCnEkOK'
response = urlopen(download_link)
filename = response.headers
print(filename)

In [ ]:
download_link = 'https://drive.google.com/uc?export=download&id=1Y3R1nu1uO66VD4RmXHGhiKMqztCnEkOK'

In [ ]:
webbrowser.open(download_link)

In [ ]:
import urllib.request
urllib.request.urlretrieve(download_link,'.obj')

In [ ]:
import requests

api = 'msy_nkfrIXgu8woN4Ei9bgs3MJ3QcMISwdvhNsTK'
download_link = 'https://2015.filemail.com/api/file/get?filekey=xmHqeumxh8Qe-lICNOFw-G-sLemA5uC0NWZ2Nchf3R7Mq35TCtD-bZ9n&pk_vid=dbb08defd6f73cdf17217318673f7d58'
payload = {
    "model_url": f"{download_link}",
    "object_prompt": "a monster mask",
    "style_prompt": "red fangs, Samurai outfit that fused with japanese batik style",
    "enable_original_uv": True,
    "enable_pbr": False,
    "resolution": "2048",
    "negative_prompt": "low quality, low resolution, low poly, ugly"
}
headers = {
    "Authorization": f"Bearer {api}"
}

response = requests.post(
    "https://api.meshy.ai/v1/text-to-texture",
    headers=headers,
    json=payload,
)
response.raise_for_status()
print(response.json())

while True:
    print()


In [ ]:
import requests

headers = {
    "Authorization": f"Bearer {api}"
}

response = requests.get(
    f"https://api.meshy.ai/v1/text-to-texture/{task_id}",
    headers=headers,
)
response.raise_for_status()
print(response.json())


In [ ]:
response.json()['texture_urls']['base_color']

In [ ]:
response.json()

In [ ]:


import urllib.request
urllib.request.urlretrieve("https://assets.meshy.ai/aae3bece-62bf-4850-b451-26f3ed655929/tasks/0190deca-b556-704f-bed1-0964471a348a/output/preview.png?Expires=1721984010&Signature=TDnpe0kcjUEroofzgCwfGeY9kA3s6jaNTJPhN-8gzxe3wsdUF-gIHae~8UZ06KTCjN~nWKLvoZfLm1559TC0y9lxQ7jLEdsueqvcHavOpSg0152XqCAvYwIS8DCzETgps1MToS4uPrYqTyONynwJmyD5MeYnj995yAOhJ2bvctfM4dAgHewDsxNvyQ9Jp1-gdds-IgrtLC7wIeKaaDofeMGYRY4X7MDNawq8da5SDCldbDrN-PIX0C2QT62ROdr5HKl9hLugqTZdO5RUlGLzCmXbd52PjYDkJxBRIjy3sokpvhqDaKNyFqre5TvQXlK7o4KaN3S39Rrb4F-hvHo7cw__&Key-Pair-Id=KL5I0C8H7HX83", "thumbnail.png")

In [ ]:
# Use meshy api to generate the texture

# header请求头
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36',
  'Cookie': cookies # 填入网盘账号的cookie
}

# 调用网盘api(method=download即下载，还有很多其他方法，请参考百度开发者api)
url = 'https://pcs.baidu.com/rest/2.0/pcs/file?method=download&app_id=309847&path=/[网盘文件路径]/xxx文件'

# 发送get请求
response = requests.get(url, headers=headers)

# 响应状态码
print(response.status_code)

# 响应内容
print(response.text)
 

In [ ]:
from PIL import  Image
img = Image.open('meshy_texture/Image_0.jpg')
c_mesh  = color_the_mesh(combined_mesh,img,combined_mesh.visual.uv)

In [ ]:
c_mesh.visual.kind

In [ ]:
c_mesh.visual = c_mesh.visual.to_color()
c_mesh.visual.vertex_colors

In [ ]:
import open3d as o3d
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
verts = np.asarray(c_mesh.vertices)
color = np.asarray(c_mesh.visual.vertex_colors)
colors = color[:, :3] / 255.0


In [ ]:

# # Extract coordinates and labels
# points = pcd[:,:3]# # Extract coordinates and labels
# points = pcd[:,:3]
# labels = pcd[:,3]
# # Normalize labels for coloring
# max_label = labels.max()
# colors = plt.get_cmap("viridis")(labels / max_label)[:, :3]

# Create Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(verts)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd], window_name="Point Cloud with Segmentation Labels", width=800, height=600)
# colors = plt.get_cmap("viridis")(labels / max_label)[:, :3]

# Create Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(verts)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd], window_name="Point Cloud with Segmentation Labels", width=800, height=600)

In [ ]:
np.asarray(cabniet.faces).astype('uint32') == indices
# cabniet faces has been changed

In [ ]:
lables = point_cloud_[:,3]


In [ ]:
indices_new = cabniet.faces
indices_old = indices
differing_indices = []
old_labels = lables 
new_labels = copy.deepcopy(labels)

for i in range(indices_one.shape[0]):
    for j in range(indices_one.shape[1]):
        if indices_one[i, j] != indices_two[i, j]:
            differing_indices.append((i, j, indices_one[i, j], indices_two[i, j]))
            new_labels[indices_one[i, j]] = old_labels[indices_two[i,j]]
            

# # Display differing indices
# for diff in differing_indices:
#     print(f"Difference at row {diff[0]}, column {diff[1]}: indices_one={diff[2]}, indices_two={diff[3]}")

In [ ]:
len(new_labels),len(old_labels)

In [ ]:


# Create Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd], window_name="Point Cloud with Segmentation Labels", width=800, height=600)